In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
tf.enable_eager_execution()
tf.random.set_random_seed(13)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU
from keras.optimizers import SGD
from keras import regularizers
import math
from sklearn.metrics import mean_squared_error
import os

Using TensorFlow backend.


In [4]:
!pip install wget

path = './drive/My Drive/Stock_price/data/btc_1d'

import numpy as np
import pandas as pd
import wget

# pull some data
url = 'http://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv'
table = wget.download(url)

In [0]:
data = pd.read_csv('Coinbase_BTCUSD_1h.csv', header = 1, index_col = 'Date')
data = data.reindex(index = data.index[::-1])
!rm Coinbase_BTCUSD_1h.csv

In [6]:
data

,Symbol,Open,High,Low,Close,Volume BTC,Volume USD
Date,,,,,,,
2017-07-01 11-AM,BTCUSD,2505.56,2513.38,2495.12,2509.17,114.60,287000.32
2017-07-01 12-PM,BTCUSD,2509.17,2512.87,2484.99,2488.43,157.36,393142.50
2017-07-01 01-PM,BTCUSD,2488.43,2488.43,2454.40,2454.43,280.28,693254.01
2017-07-01 02-PM,BTCUSD,2454.43,2473.93,2450.83,2459.35,289.42,712864.80
2017-07-01 03-PM,BTCUSD,2459.35,2475.00,2450.00,2467.83,276.82,682105.41
...,...,...,...,...,...,...,...
2019-11-28 03-PM,BTCUSD,7464.11,7647.85,7460.70,7621.76,1252.14,9493665.21
2019-11-28 04-PM,BTCUSD,7621.76,7624.00,7545.81,7616.27,1046.33,7925769.91
2019-11-28 05-PM,BTCUSD,7616.27,7627.15,7580.41,7590.09,321.86,2445797.90


In [0]:
def heikin_ashi(df):
    heikin_ashi_df = pd.DataFrame(index=df.index.values, columns=['Open', 'High', 'Low', 'Close'])
    
    heikin_ashi_df['Close'] = (df['Open'] + df['High'] + df['Low'] + df['Close']) / 4
    
    for i in range(len(df)):
        if i == 0:
            heikin_ashi_df.iat[0, 0] = df['Open'].iloc[0]
        else:
            heikin_ashi_df.iat[i, 0] = (heikin_ashi_df.iat[i-1, 0] + heikin_ashi_df.iat[i-1, 3]) / 2
        
    heikin_ashi_df['High'] = heikin_ashi_df.loc[:, ['Open', 'Close']].join(df['High']).max(axis=1)
    
    heikin_ashi_df['Low'] = heikin_ashi_df.loc[:, ['Open', 'Close']].join(df['Low']).min(axis=1)
    
    return heikin_ashi_df

In [0]:
ha = heikin_ashi(data)

In [9]:
ha

,Open,High,Low,Close
2017-07-01 11-AM,2505.56,2513.380000,2495.120000,2505.8075
2017-07-01 12-PM,2505.68,2512.870000,2484.990000,2498.8650
2017-07-01 01-PM,2502.27,2502.274375,2454.400000,2471.4225
2017-07-01 02-PM,2486.85,2486.848437,2450.830000,2459.6350
2017-07-01 03-PM,2473.24,2475.000000,2450.000000,2463.0450
...,...,...,...,...
2019-11-28 03-PM,7470.5,7647.850000,7460.700000,7548.6050
2019-11-28 04-PM,7509.55,7624.000000,7509.553290,7601.9600
2019-11-28 05-PM,7555.76,7627.150000,7555.756645,7603.4800
2019-11-28 06-PM,7579.62,7643.150000,7579.618322,7603.8525


In [0]:
def cumcnt_indices(v):
    v[~v] = pd.np.nan
    cumsum = v.cumsum().fillna(method='pad')
    reset = -cumsum[v.isnull()].diff().fillna(cumsum)
    r = v.where(v.notnull(), reset).cumsum().fillna(0.0)
    return r.astype(int)

def td_sequential(close):
    close4 = close.shift(4)
    td = cumcnt_indices(close > close4)
    ts = cumcnt_indices(close < close4)
    return td, ts

def get_technical_indicators(dataset):
    # Create 7 and 21 days Moving Average
    #dataset['ma7'] = dataset['Close'].rolling(window=7).mean()
    #dataset['ma21'] = dataset['Close'].rolling(window=21).mean()
    
    # Create MACD
    dataset['26ema'] = dataset['Close'].ewm(span=26).mean()
    dataset['12ema'] = dataset['Close'].ewm(span=12).mean()
    dataset['MACD'] = (dataset['12ema']-dataset['26ema'])
    
    # Create Exponential moving average
    #dataset['ema'] = dataset['Close'].ewm(com=0.5).mean()

    return dataset

tdup,tddn = td_sequential(ha['Close'])
ha['tdup'] = [(i if 0<i<10 else 0) for i in tdup]
ha['tddn'] = [(i if 0<i<10 else 0) for i in tddn]

ha['td'] = ha['tdup'] - ha['tddn']
ha = get_technical_indicators(ha)
ha = ha.drop(columns = ['tdup', 'tddn'])

In [11]:
ha.head()

,Open,High,Low,Close,td,26ema,12ema,MACD
2017-07-01 11-AM,2505.56,2513.380000,2495.12,2505.8075,0,2505.807500,2505.807500,0.000000
2017-07-01 12-PM,2505.68,2512.870000,2484.99,2498.8650,0,2502.202740,2502.046979,-0.155761
2017-07-01 01-PM,2502.27,2502.274375,2454.40,2471.4225,0,2491.143699,2490.094238,-1.049461
2017-07-01 02-PM,2486.85,2486.848437,2450.83,2459.6350,0,2482.335244,2480.479447,-1.855797
2017-07-01 03-PM,2473.24,2475.000000,2450.00,2463.0450,-1,2477.861757,2475.742570,-2.119187


In [12]:
ha.isna().any()

Open     False
High     False
Low      False
Close    False
td       False
26ema    False
12ema    False
MACD     False
dtype: bool

In [0]:
ha = ha.drop(columns=['Open', 'High', 'Low', 'Close', '26ema', '12ema'])

In [0]:
reg = pd.DataFrame(data['Close'])

In [0]:
periods = 10
thresh_long = .1
thresh_short = -.1

In [0]:
target = reg.pct_change(periods = periods)

In [17]:
target

,Close
Date,
2017-07-01 11-AM,NaN
2017-07-01 12-PM,NaN
2017-07-01 01-PM,NaN
2017-07-01 02-PM,NaN
2017-07-01 03-PM,NaN
...,...
2019-11-28 03-PM,0.021785
2019-11-28 04-PM,0.012985
2019-11-28 05-PM,0.011295


In [0]:
target = target[periods:]

In [19]:
target[np.logical_and(target < thresh_long, target > thresh_short)] = 0
target[target >= thresh_long] = 1
target[target <= thresh_short] = -1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3530: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [20]:
target.tail()

,Close
Date,
2019-11-28 03-PM,0.0
2019-11-28 04-PM,0.0
2019-11-28 05-PM,0.0
2019-11-28 06-PM,0.0
2019-11-28 07-PM,0.0


In [0]:
# ha -> features
# reg -> price

single = pd.concat([reg], axis=1)

scm = MinMaxScaler(feature_range=(-1, 1))
single = scm.fit_transform(single)

In [0]:
X = []
y = []
lookback = 20
future = 1

for i in range(single.shape[0]-lookback-future):
    slice_x = np.array(single[i:lookback+i]) 
    slice_y = reg['Close'].values[lookback+i+future]
    X.append(slice_x)
    y.append(slice_y)

X = np.array(X)
y = np.array(y).reshape(-1,1)
y = scm.fit_transform(y)

In [67]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.shape, X_test.shape, reg.shape

((15831, 20, 1), (15831, 1))

In [69]:
y_train.shape, y_test.shape

((15831, 1), (5277, 1))

In [0]:
weight_decay = 0.00005
batch_size = 256

UV = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32,return_sequences = False, input_shape = X_train.shape[-2:]),
    #tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])
sgd = tf.keras.optimizers.SGD(lr=1e-2, momentum = .8, decay = 1e-6, nesterov = False)
UV.compile(optimizer = 'adam',loss='mae')

In [71]:
UV.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 32)                3264      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 3,297
Trainable params: 3,297
Non-trainable params: 0
_________________________________________________________________


In [0]:
UV.load_weights(os.path.join(path, 'UV.h5'))

In [0]:
UV.fit(X_train, y_train, epochs = 100, 
          validation_data = (X_test, y_test), batch_size = batch_size)

In [0]:
UV.save_weights(os.path.join(path, 'UV.h5'))

In [0]:
# Get last 20 entries
val = pd.concat([reg], axis=1)
val = val.iloc[-lookback:]
val = scm.fit_transform(val)
val = val.reshape(1,lookback,1)
y_pred = scm.inverse_transform(UV.predict(val))

In [78]:
y_pred

array([[7617.974]], dtype=float32)

### Multivariate

In [0]:
multi = pd.concat([ha, reg], axis=1)

scm = MinMaxScaler(feature_range=(-1, 1))
multi = scm.fit_transform(multi)

In [0]:
X = []
y = []
lookback = 20
future = 1

for i in range(multi.shape[0]-lookback-future):
    slice_x = np.array(multi[i:lookback+i]) 
    slice_y = reg['Close'].values[lookback+i+future]
    X.append(slice_x)
    y.append(slice_y)

X = np.array(X)
y = np.array(y).reshape(-1,1)
y = scm.fit_transform(y)

In [81]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
X_train.shape, y_train.shape

((15831, 20, 3), (15831, 1))

In [0]:
weight_decay = 0.00005
batch_size = 256

MV = tf.keras.models.Sequential([
    tf.keras.layers.GRU(32,return_sequences = False, input_shape = X_train.shape[-2:]),
    #tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])
sgd = tf.keras.optimizers.SGD(lr=1e-2, momentum = .8, decay = 1e-6, nesterov = False)
MV.compile(optimizer = 'adam',loss='mae')

In [83]:
MV.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_6 (GRU)                  (None, 32)                3456      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 3,489
Trainable params: 3,489
Non-trainable params: 0
_________________________________________________________________


In [0]:
MV.load_weights(os.path.join(path, 'MV.h5'))

In [0]:
MV.fit(X_train, y_train, epochs = 100, 
          validation_data = (X_test, y_test), batch_size = batch_size)

In [0]:
MV.save_weights(os.path.join(path, 'MV.h5'))

In [0]:
# Get last 20 entries
val = pd.concat([ha, reg], axis=1)
val = val.iloc[-lookback:]
val = scm.fit_transform(val)
val = val.reshape(1,lookback,3)
print(val.shape)
y_pred = scm.inverse_transform(MV.predict(val))

In [0]:
y_pred